# Validation plot for photometry paper

In [ ]:
import glob
import numpy as np 
from astropy.io import fits
from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *

### To Do: 
#### - Get image of isolated source
#### - Run tractor on image
#### - Plot with heasarc region files, colorbar for chi

In [ ]:
file_location = '/media/bethany/Seagate Portable Drive/SwiftData/Astrometry/SMC/sw00040415001um2_sk_40415_1_1'
HDR = fits.open(file_location+'.new')
CAT = file_location+'.full.dat'
uvfilter='UVM2'
#pd.read_csv(file_location+'.full.dat',delimiter='\s+',names=['RAhr','DEdeg','Umag','e_Umag','Bmag','e_Bmag','Vmag','e_Vmag','Imag','e_Imag','Flag','Jmag','e_Jmag','Hmag','e_Hmag','Ksmag','e_Ksmag'])

# Step 1: Get Zaritsky Coordinates and Catalog 
shape = np.shape(HDR[0].data)

print('---Getting Catalog---')

meta = get_meta().with_hdu(HDR[0],
                           optical_catalog=CAT,
                           xdim=[int(shape[1]/2)-50,int(shape[1]/2)+50],
                           ydim=[int(shape[0]/2)-50,int(shape[0]/2)+50])

print('---Getting Background---')
bkgd = BkgdEstimator(meta,n_pix = [20,20])


print('---Getting PSF---')
pix_scale = np.abs(meta.cdelt)*3600.

psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                     uvfilter = uvfilter, width = 23).psf


print('---Running Tractor---')

fwhm = 2.5
sigma =  fwhm/2.355
# Restrict by a factor of 4.
sigma = sigma/4.
        
TractorObject = PhotometryTools(meta,
                    psf_filename = psf_object,
                    fits_table_sname = f"temp_tractor.fits", 
                    background = bkgd,
                    fit_positions = sigma,
                    threshold = 1.5)